# Cross-Country Solar Potential Comparison

## Overview
This notebook synthesizes cleaned datasets from Benin, Sierra Leone, and Togo to identify relative solar potential and key differences across countries. The analysis will inform strategic solar investment decisions for MoonLight Energy Solutions.

## Objectives
1. Load and compare cleaned datasets from all three countries
2. Visual comparison using boxplots for GHI, DNI, DHI
3. Statistical summary table (mean, median, standard deviation)
4. Statistical testing to assess significance of differences
5. Key observations and insights
6. Country ranking by solar potential

## Countries Analyzed
- **Benin (Malanville)**
- **Sierra Leone (Bumbuna)**
- **Togo (Dapaong)**


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, kruskal
import warnings
import os

warnings.filterwarnings('ignore')

# Set plotting style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('ggplot')

sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


In [ ]:
# Set working directory to project root
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    os.chdir('..')
elif not os.path.exists('data'):
    os.chdir('..')

print(f"Current working directory: {os.getcwd()}")

# Load cleaned datasets from all three countries
print("\nLoading cleaned datasets...")
benin_df = pd.read_csv('data/benin_clean.csv')
sierraleone_df = pd.read_csv('data/sierraleone_clean.csv')
togo_df = pd.read_csv('data/togo_clean.csv')

# Convert Timestamp to datetime and set as index
for df in [benin_df, sierraleone_df, togo_df]:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)

# Add country identifier to each dataframe
benin_df['Country'] = 'Benin'
sierraleone_df['Country'] = 'Sierra Leone'
togo_df['Country'] = 'Togo'

print(f"\n✅ Benin dataset loaded: {benin_df.shape}")
print(f"✅ Sierra Leone dataset loaded: {sierraleone_df.shape}")
print(f"✅ Togo dataset loaded: {togo_df.shape}")

# Display basic info
print(f"\nBenin date range: {benin_df.index.min()} to {benin_df.index.max()}")
print(f"Sierra Leone date range: {sierraleone_df.index.min()} to {sierraleone_df.index.max()}")
print(f"Togo date range: {togo_df.index.min()} to {togo_df.index.max()}")


## 1. Metric Comparison - Boxplots


In [ ]:
# Combine all dataframes for easier comparison
all_countries = pd.concat([benin_df, sierraleone_df, togo_df], ignore_index=False)

# Create boxplots for GHI, DNI, DHI side-by-side
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# GHI Boxplot
sns.boxplot(data=all_countries, x='Country', y='GHI', ax=axes[0], palette=['#FF6B35', '#004E89', '#06A77D'])
axes[0].set_title('Global Horizontal Irradiance (GHI) Comparison', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Country', fontsize=12)
axes[0].set_ylabel('GHI (W/m²)', fontsize=12)
axes[0].grid(True, alpha=0.3, axis='y')

# DNI Boxplot
sns.boxplot(data=all_countries, x='Country', y='DNI', ax=axes[1], palette=['#FF6B35', '#004E89', '#06A77D'])
axes[1].set_title('Direct Normal Irradiance (DNI) Comparison', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Country', fontsize=12)
axes[1].set_ylabel('DNI (W/m²)', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='y')

# DHI Boxplot
sns.boxplot(data=all_countries, x='Country', y='DHI', ax=axes[2], palette=['#FF6B35', '#004E89', '#06A77D'])
axes[2].set_title('Diffuse Horizontal Irradiance (DHI) Comparison', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Country', fontsize=12)
axes[2].set_ylabel('DHI (W/m²)', fontsize=12)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Boxplots created for GHI, DNI, and DHI across all three countries.")


## 2. Summary Statistics Table


In [ ]:
# Create summary statistics table for GHI, DNI, DHI
metrics = ['GHI', 'DNI', 'DHI']
countries = ['Benin', 'Sierra Leone', 'Togo']
dataframes = [benin_df, sierraleone_df, togo_df]

# Initialize summary dictionary
summary_data = []

for metric in metrics:
    for country, df in zip(countries, dataframes):
        summary_data.append({
            'Metric': metric,
            'Country': country,
            'Mean': df[metric].mean(),
            'Median': df[metric].median(),
            'Std Dev': df[metric].std(),
            'Min': df[metric].min(),
            'Max': df[metric].max()
        })

summary_df = pd.DataFrame(summary_data)

# Pivot for better readability
summary_pivot = summary_df.pivot(index='Country', columns='Metric', values=['Mean', 'Median', 'Std Dev'])

print("Summary Statistics Table:")
print("=" * 100)
print(summary_pivot.round(2))

# Also create a formatted table for each metric
print("\n" + "=" * 100)
print("Detailed Comparison by Metric:")
print("=" * 100)

for metric in metrics:
    print(f"\n{metric} Statistics:")
    print("-" * 80)
    metric_summary = summary_df[summary_df['Metric'] == metric][['Country', 'Mean', 'Median', 'Std Dev']]
    print(metric_summary.to_string(index=False))
    print()


## 3. Statistical Testing


In [ ]:
# Statistical Testing: One-way ANOVA and Kruskal-Wallis test
print("Statistical Testing Results")
print("=" * 80)

# Prepare data for testing
ghi_benin = benin_df['GHI'].dropna()
ghi_sierraleone = sierraleone_df['GHI'].dropna()
ghi_togo = togo_df['GHI'].dropna()

# Check normality assumption (Shapiro-Wilk test on samples)
print("\n1. Normality Check (Shapiro-Wilk test on samples):")
print("-" * 80)
sample_size = min(5000, len(ghi_benin))  # Use sample for performance

_, p_benin = stats.shapiro(ghi_benin.sample(sample_size))
_, p_sierraleone = stats.shapiro(ghi_sierraleone.sample(sample_size))
_, p_togo = stats.shapiro(ghi_togo.sample(sample_size))

print(f"Benin GHI normality p-value: {p_benin:.6f}")
print(f"Sierra Leone GHI normality p-value: {p_sierraleone:.6f}")
print(f"Togo GHI normality p-value: {p_togo:.6f}")

# Since data is likely not normally distributed (large sample, likely skewed), use Kruskal-Wallis
print("\n2. Kruskal-Wallis Test (Non-parametric alternative to ANOVA):")
print("-" * 80)
h_statistic, p_value_kruskal = kruskal(ghi_benin, ghi_sierraleone, ghi_togo)
print(f"H-statistic: {h_statistic:.4f}")
print(f"P-value: {p_value_kruskal:.6f}")

if p_value_kruskal < 0.05:
    print("✅ Result: Statistically significant difference between countries (p < 0.05)")
else:
    print("❌ Result: No statistically significant difference between countries (p >= 0.05)")

# Also perform ANOVA for comparison (even if assumptions may not be fully met)
print("\n3. One-way ANOVA Test:")
print("-" * 80)
f_statistic, p_value_anova = f_oneway(ghi_benin, ghi_sierraleone, ghi_togo)
print(f"F-statistic: {f_statistic:.4f}")
print(f"P-value: {p_value_anova:.6f}")

if p_value_anova < 0.05:
    print("✅ Result: Statistically significant difference between countries (p < 0.05)")
else:
    print("❌ Result: No statistically significant difference between countries (p >= 0.05)")

print("\n" + "=" * 80)
print("Interpretation:")
print(f"- Kruskal-Wallis p-value: {p_value_kruskal:.6f}")
print(f"- ANOVA p-value: {p_value_anova:.6f}")
print("\nNote: Kruskal-Wallis is preferred for non-normal distributions.")
print("Both tests indicate whether there are significant differences in GHI across countries.")


## 4. Key Observations


# Calculate key statistics for observations
ghi_stats = {}
for country, df in zip(['Benin', 'Sierra Leone', 'Togo'], [benin_df, sierraleone_df, togo_df]):
    ghi_stats[country] = {
        'median': df['GHI'].median(),
        'mean': df['GHI'].mean(),
        'std': df['GHI'].std(),
        'max': df['GHI'].max()
    }

# Find country with highest median GHI
highest_median_country = max(ghi_stats.items(), key=lambda x: x[1]['median'])
highest_median_std = highest_median_country[1]['std']
highest_median_value = highest_median_country[1]['median']

# Find most consistent (lowest std dev)
most_consistent = min(ghi_stats.items(), key=lambda x: x[1]['std'])
most_variable = max(ghi_stats.items(), key=lambda x: x[1]['std'])

# Determine variability level
variability_level = "high" if highest_median_std > 200 else "moderate" if highest_median_std > 150 else "low"

print("### Key Insights from Cross-Country Comparison:")
print()
print(f"1. **Solar Potential Ranking**: {highest_median_country[0]} shows the highest median GHI of {highest_median_value:.2f} W/m², indicating the strongest solar potential. However, it also exhibits {variability_level} variability (std dev: {highest_median_std:.2f} W/m²), suggesting {'that while peak performance is high, there may be significant day-to-day variation' if variability_level == 'high' else 'relatively stable solar conditions' if variability_level == 'low' else 'moderate variability in solar output'}.")
print()
print(f"2. **Consistency vs. Peak Performance**: {most_consistent[0]} demonstrates more consistent solar irradiance (std dev: {most_consistent[1]['std']:.2f} W/m²) compared to {most_variable[0]} (std dev: {most_variable[1]['std']:.2f} W/m²), which shows {'higher peak values but' if most_variable[1]['max'] > most_consistent[1]['max'] else ''} greater variability. This suggests {most_consistent[0]} may be more reliable for consistent energy generation, while {most_variable[0]} offers {'higher potential during optimal conditions' if most_variable[1]['max'] > most_consistent[1]['max'] else 'similar peak potential with more variability'}.")
print()
sig_status = "significant" if p_value_kruskal < 0.05 else "non-significant"
print(f"3. **Statistical Significance**: The statistical tests reveal {sig_status} differences between countries (Kruskal-Wallis p-value: {p_value_kruskal:.6f}), indicating that {'the observed differences in solar irradiance are statistically meaningful and not due to chance' if p_value_kruskal < 0.05 else 'while there may be visual differences, they are not statistically significant at the 0.05 level'}. This finding supports {'prioritizing the country with highest GHI for solar investment' if p_value_kruskal < 0.05 else 'considering other factors beyond just average GHI, such as consistency, infrastructure, and cost'}.")


In [ ]:
# Calculate average GHI for each country
ghi_means = {
    'Benin': benin_df['GHI'].mean(),
    'Sierra Leone': sierraleone_df['GHI'].mean(),
    'Togo': togo_df['GHI'].mean()
}

# Sort by average GHI
sorted_countries = sorted(ghi_means.items(), key=lambda x: x[1], reverse=True)

print("Average GHI by Country:")
print("=" * 80)
for country, avg_ghi in sorted_countries:
    print(f"{country}: {avg_ghi:.2f} W/m²")

# Create ranking bar chart
fig, ax = plt.subplots(figsize=(10, 6))

countries_ranked = [item[0] for item in sorted_countries]
ghi_values_ranked = [item[1] for item in sorted_countries]

colors = ['#FF6B35' if c == 'Benin' else '#004E89' if c == 'Sierra Leone' else '#06A77D' for c in countries_ranked]

bars = ax.bar(countries_ranked, ghi_values_ranked, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars, ghi_values_ranked)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.2f} W/m²',
            ha='center', va='bottom', fontweight='bold', fontsize=11)

ax.set_title('Country Ranking by Average Global Horizontal Irradiance (GHI)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Country', fontsize=12)
ax.set_ylabel('Average GHI (W/m²)', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

# Add ranking numbers
for i, country in enumerate(countries_ranked, 1):
    ax.text(i-1, ghi_values_ranked[i-1] + max(ghi_values_ranked) * 0.02,
            f'#{i}', ha='center', va='bottom', fontsize=14, fontweight='bold', color='darkblue')

plt.tight_layout()
plt.show()

print(f"\n🏆 Ranking: 1st - {sorted_countries[0][0]}, 2nd - {sorted_countries[1][0]}, 3rd - {sorted_countries[2][0]}")


## 5. Additional Comparative Analysis


In [ ]:
# Additional comparison: Temperature and other weather variables
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Temperature comparison
sns.boxplot(data=all_countries, x='Country', y='Tamb', ax=axes[0, 0], palette=['#FF6B35', '#004E89', '#06A77D'])
axes[0, 0].set_title('Ambient Temperature Comparison', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Country', fontsize=11)
axes[0, 0].set_ylabel('Temperature (°C)', fontsize=11)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Relative Humidity comparison
sns.boxplot(data=all_countries, x='Country', y='RH', ax=axes[0, 1], palette=['#FF6B35', '#004E89', '#06A77D'])
axes[0, 1].set_title('Relative Humidity Comparison', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Country', fontsize=11)
axes[0, 1].set_ylabel('Relative Humidity (%)', fontsize=11)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Wind Speed comparison
sns.boxplot(data=all_countries, x='Country', y='WS', ax=axes[1, 0], palette=['#FF6B35', '#004E89', '#06A77D'])
axes[1, 0].set_title('Wind Speed Comparison', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Country', fontsize=11)
axes[1, 0].set_ylabel('Wind Speed (m/s)', fontsize=11)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Barometric Pressure comparison
sns.boxplot(data=all_countries, x='Country', y='BP', ax=axes[1, 1], palette=['#FF6B35', '#004E89', '#06A77D'])
axes[1, 1].set_title('Barometric Pressure Comparison', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Country', fontsize=11)
axes[1, 1].set_ylabel('Barometric Pressure (hPa)', fontsize=11)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 6. Summary & Recommendations


In [ ]:
# Final summary comparison
print("=" * 100)
print("CROSS-COUNTRY SOLAR POTENTIAL COMPARISON SUMMARY")
print("=" * 100)

print(f"\n📊 Dataset Overview:")
print(f"   • Benin: {len(benin_df):,} records")
print(f"   • Sierra Leone: {len(sierraleone_df):,} records")
print(f"   • Togo: {len(togo_df):,} records")

print(f"\n☀️  Average GHI Ranking:")
for i, (country, avg_ghi) in enumerate(sorted_countries, 1):
    print(f"   {i}. {country}: {avg_ghi:.2f} W/m²")

print(f"\n📈 Statistical Test Results:")
print(f"   • Kruskal-Wallis p-value: {p_value_kruskal:.6f}")
print(f"   • ANOVA p-value: {p_value_anova:.6f}")
if p_value_kruskal < 0.05:
    print(f"   • Conclusion: Significant differences exist between countries")
else:
    print(f"   • Conclusion: No significant differences detected")

# Calculate most consistent (lowest std dev) and highest peak
most_consistent_country = min(ghi_stats.items(), key=lambda x: x[1]['std'])
highest_peak_country = max(ghi_stats.items(), key=lambda x: x[1]['max'])

print(f"\n🔍 Key Metrics Comparison:")
print(f"   • Highest Average GHI: {sorted_countries[0][0]} ({sorted_countries[0][1]:.2f} W/m²)")
print(f"   • Most Consistent GHI: {most_consistent_country[0]} (std dev: {most_consistent_country[1]['std']:.2f} W/m²)")
print(f"   • Highest Peak GHI: {highest_peak_country[0]} ({highest_peak_country[1]['max']:.2f} W/m²)")

print("\n" + "=" * 100)
print("💡 Strategic Recommendations:")
print("=" * 100)
print("Based on this analysis, MoonLight Energy Solutions should consider:")
print(f"1. {sorted_countries[0][0]} for maximum energy generation potential ({sorted_countries[0][1]:.2f} W/m² average)")
print(f"2. {most_consistent_country[0]} for reliable, predictable output (lowest variability: {most_consistent_country[1]['std']:.2f} W/m²)")
print(f"3. {highest_peak_country[0]} for peak performance scenarios (peak GHI: {highest_peak_country[1]['max']:.2f} W/m²)")
print("4. Further analysis of seasonal patterns and weather variability")
print("5. Cost-benefit analysis considering installation and maintenance factors")
print("6. Risk assessment based on variability and consistency metrics")
print("\n" + "=" * 100)
